In [3]:
import mne
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

In [4]:
raw1 = mne.io.read_raw_edf('CHB-MIT/chb01_01.edf')
raw2 = mne.io.read_raw_edf('CHB-MIT/chb01_02.edf')

Extracting EDF parameters from /home/matth/python/SeizureSense/CHB-MIT/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/matth/python/SeizureSense/CHB-MIT/chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/tmp/ipykernel_1946/878268199.py:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw1 = mne.io.read_raw_edf('CHB-MIT/chb01_01.edf')
/tmp/ipykernel_1946/878268199.py:2: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw2 = mne.io.read_raw_edf('CHB-MIT/chb01_02.edf')


In [ ]:
class CHBData(Dataset):
    def __init__(self, CHB_files, segment_length):
        self.segments = []
        self.labels = []
        for file_path, info in CHB_files.items():
            file_path.load_data()
            for start, end, label in info:
                segment, lab = segment_eeg(start, end, label)
                self.segments.append(segment)
                self.labels.append(lab)

    def __len__():
        return len(CHB_files)
    
    def __getitem__(self, i):
        return np.nan

    def segment_eeg(start, end, label):
        return np.nan
    
    def preprocessing(file_path):
        raw = mne.io.read_raw_edf(file_path)
        raw.load_data()
        eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)


        freqs = (60,120)
        raw_notch = raw.copy().notch_filter(freqs=freqs, picks=eeg_picks)
        raw_notch.filter(l_freq=30, h_freq=None, fir_design='firwin', filter_length='auto', phase='zero', fir_window='hamming')
    

In [5]:
kys = {'CHB-MIT/chb01_01.edf':[(0, 3600, "interictal")], 'CHB-MIT/chb01_02.edf':[(0, 3600, "interictal")], }